In [57]:
import os
import json

In [58]:
images_path = 'datasets/kaist-rgbt/train/images'
val_txt_path = 'datasets/kaist-rgbt/val_split.txt'
cls_names_path = 'datasets/kaist-rgbt/kaist-rgbt.names'
output_path = 'utils/eval/KAIST_annotation.json'

In [59]:
images = []
annotations = []
categories = []
image_id = 1

In [60]:
# create categories section
categories = []
with open(cls_names_path, "r") as f:
    cls = f.readlines()
    for idx, cl in enumerate(cls):
        cl = cl.strip()
        categories.append({
            "id": idx+1,
            "name": cl
        })

In [61]:
categories

[{'id': 1, 'name': 'person'},
 {'id': 2, 'name': 'cyclist'},
 {'id': 3, 'name': 'people'},
 {'id': 4, 'name': 'person?'}]

In [72]:
images = []
annotations = []
annotation_id = 1
image_id = 1
with open(val_txt_path, 'r') as f:
    labels_list = [f.replace('{}/', '').replace('images', 'labels').replace('jpg', 'txt').rstrip('\n') for f in f.readlines()]
    for labels in labels_list:
        images.append({
            "id": image_id
        })
        with open(labels, 'r') as file:
            for line in file:
                parts = line.strip().split()
                category_id = int(parts[0]) + 1
                bbox = list(map(float, parts[1:5]))
                occlusion = int(parts[5])
                annotations.append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": category_id,
                        "bbox": bbox,
                        "occlusion": occlusion
                    })
                annotation_id += 1
        image_id += 1

In [75]:
images[-5:]
# annotations[-5:]

[{'id': 2504}, {'id': 2505}, {'id': 2506}, {'id': 2507}, {'id': 2508}]

In [76]:
final_dict = {
    "annotations": annotations,
    "images": images,
    "categories": categories
}

In [82]:
final_dict.keys()

dict_keys(['annotations', 'images', 'categories'])

In [83]:
with open(output_path, 'w') as json_file:
        json.dump(final_dict, json_file, indent=2)

In [ ]:
import xml.etree.ElementTree as ET
import json

def xml_to_json(xml_file):
    # XML 파일을 파싱
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # JSON 구조를 위한 딕셔너리 생성
    json_data = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "person"},
            {"id": 1, "name": "cyclist"},
            {"id": 2, "name": "people"},
            {"id": 3, "name": "person?"}
        ]
    }

    # 이미지 정보 추출
    image_id = 0
    json_data["images"].append({
        "id": image_id,
        "im_name": root.find('filename').text,
        "height": int(root.find('size/height').text),
        "width": int(root.find('size/width').text)
    })

    # 객체 정보 추출
    annotation_id = 0
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        x = int(bbox.find('x').text)
        y = int(bbox.find('y').text)
        w = int(bbox.find('w').text)
        h = int(bbox.find('h').text)

        json_data["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": 0,  # "person" 클래스 ID
            "bbox": [x, y, w, h],
            "height": h,
            "occlusion": int(obj.find('occlusion').text),
            "ignore": int(obj.find('difficult').text) if obj.find('difficult') is not None else 0
        })
        annotation_id += 1

    # JSON 파일로 저장
    with open('annotation.json', 'w') as outfile:
        json.dump(json_data, outfile, indent=4)

# XML 파일 이름을 지정하고 함수를 호출
xml_to_json('path_to_your_xml_file.xml')

In [5]:
import xml.etree.ElementTree as ET
import json
import os

def xml_folder_to_json(folder_path):
    # JSON 구조를 위한 딕셔너리 생성
    json_data = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "person"},
            {"id": 1, "name": "cyclist"},
            {"id": 2, "name": "people"},
            {"id": 3, "name": "person?"}
        ]
    }

    name_to_id = {category["name"]: category["id"] for category in json_data["categories"]}
    image_id = 0
    annotation_id = 0

    # 폴더 내의 모든 XML 파일을 처리
    for filename in os.listdir(folder_path):
        if filename.endswith('.xml'):
            xml_file = os.path.join(folder_path, filename)
            tree = ET.parse(xml_file)
            root = tree.getroot()

            # 이미지 정보 추가
            json_data["images"].append({
                "id": image_id,
                "im_name": root.find('filename').text,
                "height": int(root.find('size/height').text),
                "width": int(root.find('size/width').text)
            })

            # 객체 정보 추출 및 추가
            for obj in root.findall('object'):
                bbox = obj.find('bndbox')
                x = int(bbox.find('x').text)
                y = int(bbox.find('y').text)
                w = int(bbox.find('w').text)
                h = int(bbox.find('h').text)

                json_data["annotations"].append({
                    "id": annotation_id,
                    "image_id": image_id,
                    "category_id": int(name_to_id.get(obj.find('name').text)), 
                    "bbox": [x, y, w, h],
                    "height": h,
                    "occlusion": int(obj.find('occlusion').text),
                    "ignore": int(obj.find('difficult').text) if obj.find('difficult') is not None else 0
                })
                annotation_id += 1

            # 다음 이미지로 넘어가기 전에 이미지 ID를 증가
            image_id += 1

    # JSON 파일로 저장
    with open('utils/eval/KAIST_annotation.json', 'w') as outfile:
        json.dump(json_data, outfile, indent=4)

# 폴더 경로를 지정하여 함수 호출
xml_folder_to_json('datasets/kaist-rgbt/train/labels-xml')

In [4]:
json_data = {
        "images": [],
        "annotations": [],
        "categories": [
            {"id": 0, "name": "person"},
            {"id": 1, "name": "cyclist"},
            {"id": 2, "name": "people"},
            {"id": 3, "name": "person?"}
        ]
    }
name_to_id = {category["name"]: category["id"] for category in json_data["categories"]}
print(name_to_id)

{'person': 0, 'cyclist': 1, 'people': 2, 'person?': 3}
